In [ ]:
pwd

## 1.导入数据

data_zh.xlsx

此文件必须含有一列列名为`text`，列中数据为普通文本。
可以补充其他相关信息列

In [ ]:
import jieba
import numpy as np
import pandas as pd
import csv

In [ ]:
xlsx_file = pd.read_excel('./data_zh.xlsx')
csv_file = xlsx_file.to_csv('./data_zh.csv', encoding='utf-8')
csv_file=pd.read_csv('./data_zh.csv', low_memory=False, encoding='utf-8')

In [ ]:
df_file=pd.DataFrame(csv_file)
df_file.head(14)

## 2. 数据预处理

### 2.1 去除非中文字符

In [ ]:
def format_str(document):
    '''去除非中文字符'''
    content_str=''
    for _char in document:
        if (_char >= u'\u4e00' and _char <= u'\u9fa5'):
            content_str = content_str + _char
        else:
            continue
    return content_str

# 读取csv文件中的text列，将所有文章构成一个list
document_list = list(df_file['text'])
chinese_list=[]
for document in document_list:
    chinese_list.append(format_str(document))  

In [ ]:
chinese_list

### 2.2 jieba分词

自定义词典dict_my.txt的格式为：

词 3 n

In [ ]:
jieba.load_userdict("./dict_my.txt")

In [ ]:
word_lists=[]
for i in range(len(chinese_list)):
    result=[]
    seg_list=jieba.cut(chinese_list[i])
    for word in seg_list:
        result.append(word)
    word_lists.append(result)
word_lists

### 2.3 导入停用词库、自定义字典

停用词库stopwords_my.txt格式为：两个停用词间换行

In [ ]:
# 导入停用词
f=open("./stopwords_my.txt", 'r', encoding='utf-8')
stop_word_list=[]
for line in f.readlines():
    stop_word_list.append(line.strip('\n'))

In [ ]:
contents_clean = []
for word_list in word_lists:
    line_clean = []
    for word in word_list:
        if word in stop_word_list:
            continue
        line_clean.append(word)
    contents_clean.append(line_clean)

In [ ]:
# 去除停用词后的文本 list
contents_clean

### 2.4 显示高频无用词
显示高频词，并将高频且无意义的词放置停用词库

In [ ]:
# 将分此后的结果合并至一个list中
all_words = []
for line in contents_clean:
    for word in line:
        all_words.append(word)
all_words

In [ ]:
# 统计词频
from collections import Counter

wordcount = Counter(all_words)
word_count = wordcount.most_common(50)

frequence_list = []
for i in range(len(word_count)):
    frequence_list.append(word_count[i][0])
frequence_list

In [ ]:
# 获取不在停用词库中的高频词
need_to_add_stopword = []
for i in frequence_list:
    if i not in stop_word_list:
        need_to_add_stopword.append(i)
    else:
        continue
print(need_to_add_stopword)

In [ ]:
# 直接将以下输出结果放至 停用词库
print("\n".join(str(i) for i in need_to_add_stopword))

### 2.5 分词后拼接

In [ ]:
word_list=[]
for i in range(len(contents_clean)):
    k_list=['']
    for j in range(len(contents_clean[i])):
        k_list[0] += (contents_clean[i][j] + ' ')
    word_list.append(k_list[0])

In [ ]:
word_list

## 3. 合并数据

### 3.1 分词结果写入df

In [ ]:
word_data=pd.DataFrame({'text1':word_list})
word_data.head()

### 3.2 拼接两个df

In [ ]:
data = df_file.join(word_data)

In [ ]:
data.head()

### 3.3 整理新的df

In [ ]:
# 重设df的列名，根据需求设置列名，‘jiebatext’为上述处理后的列
data.columns=['order','yearseries', 'year', 'title', 'text', 'jiebatext']

In [ ]:
data.head()

In [ ]:
# 删除不需要的列
del data['order']

In [ ]:
# 查看最终的df
data.head()

### 4. 结果写入csv文件并导出来excel
经测试，R语言实现结构主体模型（STM）最好输入xlsx，因为csv会因为分隔符等问题而报错

In [ ]:
data.to_csv(path_or_buf='./data_zh_done.csv', index=False, encoding='utf-8')

In [ ]:
csv_file=pd.read_csv('./data_zh_done.csv', low_memory=False, encoding='utf-8')
csv_file.to_excel('./data_zh_done.xlsx', index=False, encoding='utf-8')